In [1]:
import os

In [16]:
%pwd

'd:\\ML-project\\Brain-Tumor\\End-to-end-CNN-and-Hybrid-CNN-RF-Brain-Tumor-Detection\\research'

In [17]:
os.chdir("../")

In [18]:
%pwd

'd:\\ML-project\\Brain-Tumor\\End-to-end-CNN-and-Hybrid-CNN-RF-Brain-Tumor-Detection'

In [8]:
# Data ingestion entity type

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_source_path: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [11]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_source_path = config.local_path,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [12]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [21]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig, from_local_dataset =True):
        self.config = config
        self.from_local_dataset = from_local_dataset


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file) and not self.from_local_dataset:
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        elif not os.path.exists(self.config.local_data_file) and self.from_local_dataset:
            logger.info(f"Loading from local dataset of size: {get_size(Path(self.config.local_source_path))}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        if not self.from_local_dataset:
            unzip_path = self.config.unzip_dir
            os.makedirs(unzip_path, exist_ok=True)
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
        else:
            unzip_path = self.config.unzip_dir
            os.makedirs(unzip_path, exist_ok=True)
            with zipfile.ZipFile(self.config.local_source_path, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)



In [23]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-08-07 18:39:40,490: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-07 18:39:40,490: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-07 18:39:40,498: INFO: common: directory: artifacts created successfully]
[2023-08-07 18:39:40,501: INFO: common: directory: artifacts/data_ingestion created successfully]
[2023-08-07 18:39:40,501: INFO: 3000418827: Loading from local dataset of size: ~ 13872 KB]
